In [5]:
import requests
import polyline

import matplotlib.pyplot as plt
import contextily as ctx

import itertools

import xml.dom.minidom
from xml.etree.ElementTree import Element, SubElement, tostring

ModuleNotFoundError: No module named 'contextily'

In [6]:
!pip3.9 install contextily

Defaulting to user installation because normal site-packages is not writeable
  Using cached contextily-1.6.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached mercantile-1.2.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached rasterio-1.3.10-cp39-cp39-macosx_10_15_x86_64.whl.metadata (14 kB)
  Using cached xyzservices-2024.6.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl.metadata (3.4 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
Using cached contextily-1.6.0-py3-none-any.whl (17 kB)
Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 3.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: /Library/De

In [ ]:
DEFAULT_RADIUS_IN_METERS = 5

In [ ]:
def osrm_format(coords):
    lat, lon = coords
    return f"{lon},{lat}"

def get_route(points):
    points = ';'.join(map(osrm_format, points))
    params = {
        'geometries': 'polyline6',
    }
    
    response = requests.get(f"http://127.0.0.1:9000/route/v1/foot/{points}", params=params)
    routes = response.json()
    
    if routes['code'] != 'Ok':
        print(routes)
        return None
    
    geometry = routes['routes'][0]['geometry']
    return polyline.decode(geometry, 6)

def get_match(points):
    radiuses = ';'.join([str(DEFAULT_RADIUS_IN_METERS)] * len(points))
    points = ';'.join(map(osrm_format, points))
    
    params = {
        'geometries': 'polyline6',
        'radiuses': radiuses,
    } 
    response = requests.get(f"http://127.0.0.1:6000/match/v1/foot/{points}", params=params)
    routes = response.json()
    
    if routes['code'] != 'Ok':
        print(routes)
        return None
    
    route = [
        polyline.decode(matching['geometry'], 6) 
        if matching['confidence'] > 0.90
        else []
        for matching in routes['matchings']
    ]

    return list(itertools.chain.from_iterable(route))

def plot_gpx(route, ax):
    latitudes = [point[0] for point in route]
    longitudes = [point[1] for point in route]
    ax.plot(longitudes, latitudes, color='red', label='Route')
    ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
    ax.legend()
    ax.set_yticks([])
    ax.set_xticks([])
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False)

In [ ]:
start = (1.3094259512853295, 103.8945520144324)
stop = (1.3551814463295238, 103.77589648110123)

points = [start] + [stop]

osrm_route = get_route(points) 
matched_route = get_route(osrm_route) 

fig, ax = plt.subplots(1, 1, figsize=(12, 30))
plot_gpx(osrm_route, ax)
plt.show()